In [1]:
%cd '/home/jovyan/js_generator/src'

/home/jovyan/js_generator/src


/home/jovyan/js_generator/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%ls

GPT/               main.py*                  montage_fuzz.py*  utils/
Train_GPT.ipynb*   montage_fuzz-EDIT.ipynb*  preprocess/
Train_LSTM.ipynb*  montage_fuzz-JSC.py       preprocess_my.py
fuzz/              montage_fuzz.ipynb*       train/


In [3]:
import os, sys, shutil
import threading
import re

from subprocess import Popen, PIPE
from utils import kill_proc
from utils.config import Config
MAIN_PATH = '/home/jovyan/'
CK_FIRST = {'lines': {'percent': '63.5', 'count': '171810', 'all': '270360'}, 
             'functions': {'percent': '35.2', 'count': '36617', 'all': '104156'}}
JSC_FIRST = {'lines': {'percent': '51.5', 'count': '222199', 'all': '431062'}, 
             'functions': {'percent': '51.1', 'count': '125741', 'all': '245920'}}

In [78]:
conf = Config(MAIN_PATH+'js_generator/conf.json',MAIN_PATH)
main_path = conf.main_path+'js_generator/'
help_dir = conf.help_dir
eng_path = conf.eng_path
eng_exec = conf.eng_exec
data_dir = conf.data_dir
first_path=main_path+'CK_FIRST/'#'JSC_FIRST/'

In [79]:
eng_path

'/home/jovyan/ChakraCore/'

In [61]:
#копирование gcda в движок
with open(first_path+'_gcda_files.txt') as file:
    files = file.read().split('\n')
for file_path in files:
     #name = f.split('/')[-1]
    name = os.path.basename(file_path)  # Извлекаем имя файла
    dst = os.path.dirname(file_path)  # Извлекаем путь до файла
    #print(name, dst)
    if name != '':
        try:
            shutil.copy(first_path+name, dst)
            #print(f"Файл {first_path+name} скопирован в {dst}")
        except IOError as e:
            print(f"Ошибка при копировании файла {first_path+name}: {e}")
        except:
            print(f"Неизвестная ошибка")
    else:
        print(f"Неизвестная ошибка")

Неизвестная ошибка


In [15]:
from utils import list_dir
seed_dir = data_dir+'/seed/'
print(len(list_dir(seed_dir)))#%time 

14148


In [80]:
sh_path = MAIN_PATH+'js_generator/My_Cov.sh'
#My_Cov_zero.sh'

In [85]:
# функция запуска движка
def get_cov(js_path):
    cmd = [sh_path] + [eng_path] + [js_path]
    # print(cmd)
    try:
        proc = Popen(cmd, stdout=PIPE, stderr=PIPE)  # stdin=PIPE,cwd=seed_dir, 
        # timer = threading.Timer(50, kill_proc, [proc])
        # timer.start()  
        # proc.wait()
        stdout, stderr = proc.communicate()
        # timer.cancel()
        dct = {}
        if stderr:
            if 'Summary coverage rate:' in stderr.decode():
                err = stderr.decode().split('Summary coverage rate:')[1]
                num = re.findall(r'\d+\.\d+|\d+', err)
                dct['lines'] = {"percent": float(num[0]),"count": int(num[1]),"all": int(num[2])}
                dct['functions'] = {"percent": float(num[3]),"count": int(num[4]),"all": int(num[5])}
                # print(dct)
                return dct
        if stdout:
            if 'Summary coverage rate:' in stdout.decode():
                out = stdout.decode().split('Summary coverage rate:')[1]
                num = re.findall(r'\d+\.\d+|\d+', out)
                dct['lines'] = {"percent": float(num[0]),"count": int(num[1]),"all": int(num[2])}
                dct['functions'] = {"percent": float(num[3]),"count": int(num[4]),"all": int(num[5])}
                # print(dct)
                return dct
                # print('stdout=' + out[1].split('\n')[0] + out[2].split('\n')[0])
            # print('stdout=' + stdout.decode() +'\n')
            # if 'Overall coverage rate' in stdout.decode():
            #     #print('stdout=' + stdout.decode() +'\n')
            # #pattern = r'lines*:'
            # #stdout = str(stdout)
            # #print(re.split(pattern, str(stdout)))
            #     st = str(stdout).split('Overall coverage rate:\\n')[1].split('\\n')[:2]
            #     dct = {}
            #     for i in st:
            #         v1 = i.split(':')[1].split('%')[0]
            #         v2 = i.split(':')[1].split('%')[1].split('of')[0].replace('(','')
            #         v3 = i.split(':')[1].split('%')[1].split('of')[1].split(' ')[1]
            #         d2 = {'percent': v1, 'count': v2, 'all':v3}
            #         dct[i.split(':')[0].replace('.', '').replace(' ', '')] = d2#.replace(' ', '')
            #     #print([v.split('%') for v in dct.values()])
            #     #print(dct)
            #     return dct
            # print('returncode=' + str(proc.returncode)+'\n')
        return 0

    except Exception as err:
        # pass
        print('Popen_ERR=' + str(err)+'\n')


In [88]:
d_pred = {}
for js in list_dir(seed_dir)[50:99]:#[:20]
    # print(js)
    d = get_cov(js)
    # with open(MAIN_PATH+'res.txt','a') as res:
    #     res.write(f"File {js} set coverage: {d}")
    # print(f"File {js} set coverage: {d}")
print(f"Coverage: {d}")
    # if d: 
    #     if d_pred != {}:
    #         for key in d_pred.keys():
    #             #for key1 in d.items():#for k,v in value.items():
    #                 if d_pred[key]['percent'] < d[key]['percent']:                        
    #                     print('INCREASED', key, d_pred[key], d[key])#['!!!'], 
    #                 elif d_pred[key]['percent'] > d[key]['percent']:  
    #                     print('DECREASED', key, d_pred[key], d[key])#['!!!'], 
    #     d_pred = d
    #     print('first', d_pred)
    # else:
    #     pass
#get_cov('/home/jovyan/erokhina/My/Coverage/ChakraCore/out/Release/ /home/jovyan/erokhina/My/Coverage/seed/156177c65a441c73b0a20de342f5fef8.js')
# print('END')

Coverage: {'lines': {'percent': 63.6, 'count': 171815, 'all': 270360}, 'functions': {'percent': 35.2, 'count': 36619, 'all': 104156}}


In [125]:
import json
def get_value(json, key_path: list):
    try:
        if len(key_path) == 1:
            #print('json[key_path[0]]'+json[key_path[0]])
            return json[key_path[0]]
        else:
            return get_value(json[key_path.pop(0)], key_path)
    except Exception:
        print ("Dose not found in a.json, but there is in b.json: ", key_path)
        return -1
    
def get_jsons_diff_no_ordered(json_1, json_2, key_path: list):  
    kp = key_path
    for key in json_2.keys():
        if type(json_2[key]) == int: # worck if field is num
            if key != "start_line" and json_2[key] != 0: # filter fields to be changed AND check logic: 01->1 else 0
                nkp = kp.copy()
                nkp.append(key)
                val = get_value(json_1, nkp)                
                if val > 0: 
                    if val != json_2[key]:
                        print(key, val, json_2[key])
                    json_2[key] = 0 # change key's value using check logic 
        elif type(json_2[key]) == dict: # reiterate if field is dic
            nkp = kp.copy()
            nkp.append(key)         
            get_jsons_diff_no_ordered(json_1, json_2[key], nkp) # iterate in deeper
        else:
            print ("Unknown behaviour, need deep code check") # should never be called
    return json_2    

In [126]:
import json
import os
def get_cov2(json_path1,json_path2): 
    name = 'diff_'+json_path1.split('/')[::-1][0].split('_')[1]
    with open(json_path1) as js1, open(json_path2) as js2:
        json_1 = json.load(js1) 
        json_2 = json.load(js2) 
        try:
            json_3 = get_jsons_diff_no_ordered(json_1, json_2, [])
        except Exception:
            print(f"Something went wrong with calculatung diff:\n", traceback.format_exc())
        else:  
            with open('/home/jovyan/erokhina/My/Coverage/diff/'+name, "w") as json_file_3: 
                json.dump(json_3, json_file_3)
                print(f"Job was done") 

In [127]:
from utils import list_dir
json_dir = '/home/jovyan/erokhina/My/Coverage/json/'
l = list_dir(json_dir)#[:6]
l1 = []
for ll in l:
    if ll.endswith('.json'):
        l1.append(ll)
for i in range(len(l1)-1):
    get_cov2(l1[i],l1[i+1])

Job was done
Job was done
Job was done
Job was done
Job was done
Job was done
Job was done
execution_count 21 16
15 21 16
18 21 16
22 21 16
111 2 3
113 1 2
1076 3 4
1078 2 3
1102 4 6
1126 2 3
1128 1 2
1129 1 2
1132 1 2
1149 1 2
1150 1 2
1151 1 2
1160 1 2
1162 1 2
1163 1 2
1171 1 2
1177 1 2
1183 1 2
1188 1 2
1193 1 2
1200 1 2
1201 1 2
1202 1 2
1203 1 2
1205 1 2
execution_count 5 10
6271 5 10
6272 5 10
89 6 4
execution_count 8 4
execution_count 2 1
execution_count 2 1
execution_count 4 1
execution_count 2 1
execution_count 2 1
execution_count 3 1
execution_count 2 1
execution_count 2 1
206 8 4
208 8 4
209 8 4
210 8 4
211 8 4
212 8 4
749 10 4
750 10 4
1913 9 4
1916 10 4
1923 9 4
execution_count 123 121
49 123 121
51 123 121
105 36 1
107 36 1
108 36 1
117 40 58
119 39 57
125 39 57
Job was done
execution_count 16 21
15 16 21
18 16 21
22 16 21
111 3 2
113 2 1
1076 4 3
1078 3 2
1102 6 4
1126 3 2
1128 2 1
1129 2 1
1132 2 1
1149 2 1
1150 2 1
1151 2 1
1160 2 1
1162 2 1
1163 2 1
1171 2 1
1177 2 1